In [35]:
import requests
from bs4 import BeautifulSoup
from td_api import Account
from datetime import datetime, timedelta
from price_probability_distobution import get_pdfs_from_deltas, get_pdfs_from_marks
import requests
from bs4 import BeautifulSoup
from td_api import Account
from datetime import datetime, timedelta
from price_probability_distobution import get_pdfs_from_deltas, get_pdfs_from_marks
from scipy import stats
from scipy.integrate import quad
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

In [11]:


CAPITAL = 10000
NUM_STOCKS = 10

try:
    page = requests.get("https://research.investors.com/options-center/reports/option-volume", headers={"User-Agent": "Chrome"})
    soup = BeautifulSoup(page.text, features="lxml")

    items = soup.findAll("a", {"class": "stockRoll"})
    tickers = [item.text for item in items]
except Exception as e:
    print("Exception when gettings tickers from investors.com")
    print(e)
    exit()

try:
    acc = Account("keys.json")
except Exception as e:
    print("Exception when creating TD account")
    print(e)

profiles = []
from_date = datetime.now()
to_date = from_date + timedelta(days=8)

last_query_time = 0

profiles = pd.DataFrame(columns=["ticker", "time", "share_price", "mean", "std", "err_mean", "err_std", "sort_key"])

for ticker in tickers:
    try:
        time.sleep(max(0, 0.6 - (time.time() - last_query_time)))
        print("Getting data for", ticker)
        data = acc.get_options_chain(ticker, from_date, to_date, strike_count=50)
        if data is None:
            continue
        mark = acc.get_quotes([ticker])["mark"].iloc[0]
        last_query_time = time.time()
        print("Calculating PDF for", ticker)
        ticker_pdfs = get_pdfs_from_deltas(data, distribution=stats.logistic)
        profile = {}
        (u, s, errs) = list(ticker_pdfs.values())[0]
        profile["ticker"] = ticker
        profile["time"] = int(time.time())
        profile["share_price"] = mark
        profile["mean"] = round(u, 3)
        profile["std"] = round(s, 3)
        profile["err_mean"] = round(errs[0], 3)
        profile["err_std"] = round(errs[1], 3)
        profile["sort_key"] = u / mark - errs[0] / u

        profiles = profiles.append(profile, ignore_index=True)
    except Exception as e:
        print("Exception when getting data or gettings PDFs for", ticker)
        print(e)

profiles.sort_values("sort_key", ascending=False, inplace=True)
profiles.drop(columns=["sort_key"], inplace=True)
profiles.set_index("ticker", inplace=True)

Getting data for SPY
Calculating PDF for SPY
Getting data for EEM
Calculating PDF for EEM
Warning dropped 13 rows containing NaN
Getting data for QQQ
Calculating PDF for QQQ
Getting data for FXI
Calculating PDF for FXI
Warning dropped 19 rows containing NaN
Getting data for AAPL
Calculating PDF for AAPL
Warning dropped 17 rows containing NaN
Getting data for HYG
Calculating PDF for HYG
Warning dropped 27 rows containing NaN
Getting data for IWM
Calculating PDF for IWM
Warning dropped 16 rows containing NaN
Getting data for XOP
Calculating PDF for XOP
Getting data for XLF
Calculating PDF for XLF
Getting data for BAC
Calculating PDF for BAC
Warning dropped 24 rows containing NaN
Getting data for AMD
Calculating PDF for AMD
Warning dropped 5 rows containing NaN
Getting data for EWZ
Calculating PDF for EWZ
Warning dropped 24 rows containing NaN
Getting data for MSFT
Calculating PDF for MSFT
Getting data for GLD
Calculating PDF for GLD
Getting data for T
Calculating PDF for T
Getting data f

KeyError: 'sort_key'

In [37]:
profiles = profiles[~profiles.index.duplicated(keep='first')]

tickers = profiles[:10].index
positions = acc.get_positions()

sell_tickers = [t for t in positions.keys() if not t in tickers]
for ticker in sell_tickers:
    print("SELL %d %s" % (positions[ticker], ticker))
    time.sleep(max(0, 0.6 - (time.time() - last_query_time)))
    acc.sell(ticker, positions[ticker])
    last_query_time = time.time()

old_tickers = [t for t in tickers if t in positions.keys()]
for ticker in old_tickers:
    mark = profiles["share_price"][ticker]
    target_shares = CAPITAL / NUM_STOCKS / mark
    change = round(target_shares - positions[ticker])
    if change < 0:
        print("SELL %d %s" % (-change, ticker))
        acc.sell(ticker, positions[ticker])
    else:
        print("BUY %d %s" % (change, ticker))
        acc.buy(ticker, positions[ticker])

new_tickers = [t for t in tickers if not t in positions.keys()]
for ticker in new_tickers:
    mark = profiles["share_price"][ticker]
    target_shares = round(CAPITAL / NUM_STOCKS / mark)
    print("BUY %d %s" % (target_shares, ticker))
    time.sleep(max(0, 0.6 - (time.time() - last_query_time)))
    acc.buy(ticker, target_shares)
    last_query_time = time.time()

KeyError: 'positions'